# Ensemble Learning & Random Forests

## Table of Contents
### 1. [Ensemble Learning](#Ensemble-Learning)
### 1. [Voting Classifiers](#Voting-Classifiers)
### 1. [Bagging & Pasting](#Bagging-&-Pasting)
### 1. [Boosting](#Boosting)
### 1. [Stacking](#Stacking)

## Ensemble Learning

Ensemble learning is when you **put together** trained models instead of picking just one.

### There are four main ensemble methods:
### 1. Voting
### 2. Bagging/Pasting
### 3. Boosting
### 4. Stacking

## Voting Classifiers

- Train **many models** and select the best
- Let the models *'vote'* on the classification
- The **majority**, or most agreed on answer is your classification

![voting image](./images/voting.jpg)

In [2]:
# Ensemble Model for Voting
from sklearn.ensemble import VotingClassifier

# Individual Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [4]:
# Three Individual Classification Models
logistic_classifier = LogisticRegression()
random_forest_classifier = RandomForestClassifier()
support_vector_classifier = SVC()

In [6]:
# One Voting Classifier to Aggregate Them
voting_classifier = VotingClassifier(
    estimators = [
        ('lr', logistic_classifier), 
        ('rf', random_forest_classifier), 
        ('svc', support_vector_classifier)
    ],
    voting = 'hard' # hard OR soft
)

## Two Types of Voting
### Hard
- Take the highest number of votes

### Soft
- Add up the probabilities to pick the one that we are most sure about

## Bagging & Pasting

## Random Forests

## Boosting

## Stacking